# Implementation of a Blind Signature Scheme with RSA

## RSA Key Pair Generation

In [143]:
import random

#Miller Rabin Primality Test
def is_prime(n, k=5):
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0:
        return False

    r, d = 0, n - 1
    while d % 2 == 0:
        d //= 2
        r += 1

    #check if a base a indicates n is composite.
    def check_composite(a):
        x = pow(a, d, n)
        if x == 1 or x == n - 1:
            return False
        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                return False
        return True

    for _ in range(k):
        a = random.randint(2, n - 2)
        if check_composite(a):
            return False
    return True

# Generates a large prime number with a specified number of bits.
def generate_large_prime(bits=32):
    while True:
        p = random.getrandbits(bits) | 1  # Ensures p is odd
        if is_prime(p):
            return p

In [144]:
p = generate_large_prime(32)
q = generate_large_prime(32)
n = p*q # prime composite
print('Prime composite n:',n)
phi_n = (p-1)*(q-1) # Euler Totient Function of n
print('Euler totient function of n:',phi_n)

Prime composite n: 9597602117755532057
Euler totient function of n: 9597602111337544320


### Greatest Common Divisor utility function

In [145]:
def gcd(a, b):
    if((a<0) or (b<0) or (a<b)):
        print("wrong parameter input")
        return

    while(b != 0):
        r = a % b
        a = b
        b = r
        
    return a

In [146]:
e = generate_large_prime(32)
print((1 < e) and (e < phi_n))
print(gcd(phi_n,e))
print('Public exponent:',e)

True
1
Public exponent: 4021403657


### Modulo Inverse utility function

In [147]:
# Applies the Extended Euclidean Algorithm to determine the greatest common divisor of a and b, and the coefficients for Bézout's identity.
def extended_gcd(a, b):
    if a == 0:
        return b, 0, 1
    gcd, x_prev, y_prev = extended_gcd(b % a, a)
    x_current = y_prev - (b // a) * x_prev
    y_current = x_prev
    return gcd, x_current, y_current

# Calculate the modular inverse of k modulo q using the Extended Euclidean Algorithm.
def modular_inverse(k, q):
    gcd, x, _ = extended_gcd(k, q)
    if gcd != 1:
        raise ValueError(f"Modular inverse doesn't exist for k = {k} under modulo q = {q}")
    return x % q

In [148]:
d = modular_inverse(e,phi_n)
print('Secret exponent:',d)

Secret exponent: 179607256138019513


In [149]:
print('Secret key (d):',d)
print('Public key (e,n):',e,n)

Secret key (d): 179607256138019513
Public key (e,n): 4021403657 9597602117755532057


## Verifying Key Pair using RSA Public Key Encryption

In [150]:
import hashlib

def read_message_from_file(filename, n):
    with open(filename, 'r') as file:
        message = file.read()
        print(message)
    return int(hashlib.sha256(message.encode()).hexdigest(), 16) % n

def write_signature_to_file(filename, signature):
    with open(filename, 'w') as file:
        file.write(str(signature))
        
def read_signature_from_file(filename):
    with open(filename, 'r') as file:
        signature = int(file.read())
    return signature

read_message_file_path = 'message.txt'
write_signature_file_path = 'signature.txt'

In [151]:
m = read_message_from_file(read_message_file_path, n)
print('Plaintext :',m)
c = pow(m,e,n) # public key encryption
print('Ciphertext:',c)
m1 = pow(c,d,n) # public key decryption
print('Decryption:',m1)

Ado mama Anjula
Plaintext : 7876866907092654881
Ciphertext: 4389271405695363308
Decryption: 7876866907092654881


## Blind Signature Generation

In [152]:
k = generate_large_prime(32)
bf = pow(k,e,n) # blinding factor
print('Blinding factor:',bf)
m1 = (m*bf)%n # blinded message
print('Blinded message:',m1)
s1 = pow(m1,d,n) # signature on blinded message
print('Signature on blinded message:',s1)
write_signature_to_file(write_signature_file_path,s1)


Blinding factor: 5779815420260312904
Blinded message: 2199951294431269806
Signature on blinded message: 5111875325994362525


In [153]:
s1 = read_signature_from_file(write_signature_file_path)
invk = modular_inverse(k,n)
s = (s1*invk)%n # recovery of signature
print('Signature on original message:',s)
s0 = pow(m,d,n) # compute the signature directly
print('Signature on original message when directly computed:',s0)

Signature on original message: 7225579094839142380
Signature on original message when directly computed: 7225579094839142380


In [154]:
if(s == s0):
    print('Signature is valid')
else:
    print('Signature is invalid')

Signature is valid
